# Aerodynamic Particle Sizer (APS) Analysis

### Christopher Rapp

rapp5@purdue.edu

**Notes:** If you are confused or stuck on certain portions of the code, print out the variables you are producing while proceeding through the code to better understand the process

This code assumes _Set# has manually been appended to the end of the exported csv files from AIM
It additionally assumes that dW, column wise, and comma seperated are the export parameters

---

**SECTION:** Import Libraries

In [1]:
import os
import re
import numpy as np
import pandas as pd
from datetime import datetime
from numpy import linspace
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from scipy.optimize import curve_fit


**SECTION:** Define working directories

In [2]:
# Location of files necessary
import_APS = '/Users/christopherrapp/Documents/Purdue/Data_SPIDER/calibrations/import/l_pcvi/APS/'

# Location to send results
export_results = '/Users/christopherrapp/Documents/Purdue/Data_SPIDER/calibrations/export/l_pcvi/'

# Location to send plots
export_plots = '/Users/christopherrapp/Documents/Purdue/Data_SPIDER/calibrations/plots/'


**SECTION:** List directories

In [3]:
APS_directories = [d for d in os.listdir(import_APS) if not(d.startswith('.'))]

APS_paths = [import_APS + k + "/csv_files/" for k in APS_directories]

APS_files = [os.listdir(f) for f in APS_paths]
APS_files

[['20210810_1348_PF45_AF6_7_SF6_5_AIF5_glassbeads_Set3.TXT',
  '20210810_1456_PF45_AF7_0_SF6_5_AIF5_glassbeads_Set6.TXT',
  '20210810_1250_PF45_AF6_5_SF6_5_AIF5_glassbeads_Set1.TXT',
  '20210810_1522_PF0_AF0_SF6_5_AIF5_glassbeads_Set7.TXT',
  '20210810_1535_PF0_AF0_SF6_5_AIF5_glassbeads_Set8.TXT',
  '20210810_1326_PF0_AF0_SF6_5_AIF5_glassbeads_Set1.TXT',
  '20210810_1335_PF45_AF6_6_SF6_5_AIF5_glassbeads_Set2.TXT',
  '20210810_1516_PF45_AF7_1_SF6_5_AIF5_glassbeads_Set7.TXT',
  '20210810_1529_PF45_AF7_2_SF6_5_AIF5_glassbeads_Set8.TXT',
  '20210810_1550_PF0_AF0_SF6_5_AIF5_glassbeads_Set9.TXT',
  '20210810_1423_PF0_AF0_SF6_5_AIF5_glassbeads_Set4.TXT',
  '20210810_1611_PF45_AF7_5_SF6_5_AIF5_glassbeads_Set11.TXT',
  '20210810_1544_PF45_AF7_3_SF6_5_AIF5_glassbeads_Set9.TXT',
  '20210810_1449_PF0_AF0_SF6_5_AIF5_glassbeads_Set5.TXT',
  '20210810_1617_PF0_AF0_SF6_5_AIF5_glassbeads_Set11.TXT',
  '20210810_1430_PF45_AF6_9_SF6_5_AIF5_glassbeads_Set5.TXT',
  '20210810_1402_PF0_AF0_SF6_5_AIF5_glassbe

**SECTION:** Loop through directories and retrieve file information using regular expressions (regex) with module re. 

In [4]:
# Define empty dictionary
D = {}

# OPEN LOOP
for i in range(0, len(APS_files), 1):
    
    # Generate temporary variable
    x = APS_files[i]
    
    # Removes system temporary files (such as a .DStore file)
    x = [f for f in x if not(f.startswith('.'))]
    
    # Initialize dictionary
    file_information = {
        'File': [],
        'Path': [],
        'Date': [],
        'Time': [],
        'PF': [],
        'AF': [],
        'SF': [],
        'AIF': [],
        'Type': [],
        'Experiment Set #': []
    }
    
    # Second loop which goes through the items within the dated directory
    for j in range(0, len(x), 1):
        
        # Extract the necessary information from filenames
        File = x[j]
        Path = APS_paths[i] + File 
        Date = x[j][0:4] + '-' + x[j][4:6] + '-' + x[j][6:8]
        Time = x[j][9:13]
        
        # These are simple regex patterns where it looks for the literal strings PF, AF, etc. then looks for 1 or more digits with potentially a _ or . after
        # It then also looks ahead to see if there are zero or more digits after the _ or . if present
        PF = str(re.findall(r'PF\d+[_.]?\d*', x[j]))
        AF = str(re.findall(r'AF\d+[_.]?\d*', x[j]))
        SF = str(re.findall(r'SF\d+[_.]?\d*', x[j]))
        AIF = str(re.findall(r'AIF\d+[_]?\d*', x[j]))
        
        # Experiment Set
        # This part is crucial in matcing samples together for comparing inlet and outlets
        ID = str(re.findall(r'Set\d+', x[j]))

        # If else ladder
        # Missing value code is 9999 as Python is obnoxious about interpreting NA's
        if PF != "[]":
            PF = float(PF.replace('_', '.').replace('PF', '').replace("['", '').replace("']", ''))
        else:
            PF = 9999
        if AF != "[]":
            AF = float(AF.replace('_', '.').replace('AF', '').replace("['", '').replace("']", ''))
        else:
            AF = 9999
        if SF != "[]":
            SF = float(SF.replace('_', '.').replace('SF', '').replace("['", '').replace("']", ''))
        else:
            SF = 9999
            
        if AIF != "[]":
            AIF = AIF.replace('_', '.').replace('AIF', '').replace("['", '').replace("']", '')
        if AIF == '[]':
            AIF = str(re.findall(r'\d+[_]?lpm', x[j]))
        if AIF != '[]':
            AIF = float(AIF.replace('lpm', '').replace("['", '').replace("']", ''))
        else:
            AIF = 1
        if ID != '[]':
            ID = float(ID.replace('Set', '').replace("['", '').replace("']", ''))
        else:
            ID = 'NA'
            
        # Append values to dictionary
        file_information["File"].append(File)
        file_information["Path"].append(Path)
        file_information["Date"].append(Date)
        file_information["Time"].append(Time)
        file_information["PF"].append(PF)
        file_information["AF"].append(AF)
        file_information["SF"].append(SF)
        file_information["AIF"].append(AIF)
        file_information["Experiment Set #"].append(ID)
        
    # Dataframe conversion
    file_info_df = pd.DataFrame.from_dict(file_information, orient = 'index')
    D[i] = file_info_df.transpose()
    
    # CLOSE LOOP   

**SECTION:** Creates a column of type based on combination of PF, AF, and other identifiers

In [5]:
F = {}
for i in range(0, len(D), 1):
    
    # Copy the dataframe to avoid overwriting
    tmp = D[i].copy(deep = True)

    # Convert time to a float variable
    #tmp["Time"] = tmp["Time"].astype("float")
    #tmp = tmp.sort_values("Time").reset_index().drop(columns = ["index"])
    
    for j in range(0, tmp.shape[0], 1):
        
        # If else ladder of checking for the flow values in the filename
        # Substitutes values into "Type" depending on value
        if (tmp["PF"][j] == 0) & (tmp["AF"][j] == 0):
            tmp.loc[j, ["Type"]] = "Inlet"
        elif (tmp["PF"][j] == 0) & (tmp["AF"][j] != 0):
            tmp.loc[j, ["Type"]] = "AF Only"
        elif (tmp["PF"][j] == 9999) & (tmp["AF"][j] == 9999) & (tmp["SF"][j] == 9999):
            tmp.loc[j, ["Type"]] = "Other"
        elif (tmp["PF"][j] > 0) & (tmp["AF"][j] > 0):
            tmp.loc[j, ["Type"]] = "Outlet" 
    
    # Replace 9999 with numpy NaN
    tmp = tmp.replace(9999, np.nan)
            
    F[i] = tmp

Abbreviations:

* Inlet Flow (IF)
* Add Flow (AF)
* Pump Flow (PF)
* Sample Flow (SF)
* Effective Counter Flow (ECF)
* Counter Flow (CF)

PCVI Flow Equilibrium:

$$ IN = OUT $$

$$ IF + AF = PF + SF \,\,\,\,\, \Longleftrightarrow \,\,\,\,\, IF = PF + SF - AF $$

Other Definitions:

$$ ECF = $$

$$ CF = $$ 

Transmission Efficiency: $$ \frac{Outlet\ Concentration\ cm^{-3}}{Inlet\ Concentration\ cm^{-3}} \cdot \frac{Sample\ Flow\ (L\cdot\min^{-1})}{Inlet\ Flow\ (L\cdot\min^{-1})} $$


Logistic Curve (Sigmoid Curve):

$$ D(t)\ = \frac{L}{1+e^{-k(x-x_{0})}} $$

**SECTION:** Simple for loop to add categorical labels to the list of files

In [6]:
for i in range(0, len(F), 1):
    
    # Temporary variable of dataframe of files for dated directory
    X = F[i]

    for j in range(0, X.shape[0], 1):
        
        # Temporary variable for individual files in X
        y = X["File"][j]

        # If else chain using re.search to detect strings in filenames
        if re.search("test", y) != None:
            X.loc[j, ['Type']] = "Test"
        elif re.search("ambient", y) != None:
            X.loc[j, ['Type']] = "Ambient"
        elif re.search("inlet", y) != None:
            X.loc[j, ['Type']] = "Inlet"
            X.loc[j, ['PF']] = 0
            X.loc[j, ['AF']] = 0
    
    # Calculate IF
    X["IF"] = X["PF"] + X["SF"] - X["AF"]
    
    # Reassignment
    F[i] = X

In [7]:
# Print all the files available
all_files_df = pd.DataFrame()
for i in range(0, len(F), 1):    
    all_files_df = all_files_df.append(F[i])

all_files_df = all_files_df.sort_values("Date")
all_files_df

,File,Path,Date,Time,PF,AF,SF,AIF,Type,Experiment Set #,IF
0,20210714_1102_test.txt,/Users/christopherrapp/Documents/Purdue/Data_S...,2021-07-14,1102,NaN,NaN,NaN,1.0,Test,NA,NaN
2,20210714_1503_PF0_AF0_SF6_5_Set1.txt,/Users/christopherrapp/Documents/Purdue/Data_S...,2021-07-14,1503,0.0,0.0,6.5,1.0,Inlet,1,6.5
3,20210714_1440_test.txt,/Users/christopherrapp/Documents/Purdue/Data_S...,2021-07-14,1440,NaN,NaN,NaN,1.0,Test,NA,NaN
4,20210714_1451_PF35_AF5_SF6_5_Set1.txt,/Users/christopherrapp/Documents/Purdue/Data_S...,2021-07-14,1451,35.0,5.0,6.5,1.0,Outlet,1,36.5
5,20210714_1515_PF40_AF5_6_SF6_5_Set2.txt,/Users/christopherrapp/Documents/Purdue/Data_S...,2021-07-14,1515,40.0,5.6,6.5,1.0,Outlet,2,40.9
...,...,...,...,...,...,...,...,...,...,...,...
5,20210909_1320_PF45_AF6.7_SF6.5_AIF5_glassbeads...,/Users/christopherrapp/Documents/Purdue/Data_S...,2021-09-09,1320,45.0,6.7,6.5,5.0,Outlet,5,44.8
0,20210909_1304_PF0_AF0_SF6.5_AIF5_glassbeads_Se...,/Users/christopherrapp/Documents/Purdue/Data_S...,2021-09-09,1304,0.0,0.0,6.5,5.0,Inlet,2,6.5
1,20210909_1312_PF45_AF6.8_SF6.5_AIF5_glassbeads...,/Users/christopherrapp/Documents/Purdue/Data_S...,2021-09-09,1312,45.0,6.8,6.5,5.0,Outlet,4,44.7
3,20210909_1317_PF0_AF0_SF6.5_AIF5_glassbeads_Se...,/Users/christopherrapp/Documents/Purdue/Data_S...,2021-09-09,1317,0.0,0.0,6.5,5.0,Inlet,4,6.5


In [9]:
# Initialize dictionary
results = {
    'Date': [],
    'ID': [],
    'PF': [],
    'AF': [],
    'SF': [],
    'AIF': [],
    'IF': [],
    'AF:IF Ratio': [],
    'Maximum Transimission Efficiency': [],
    'D50': [],
    'D50 SD': []
}

# Loop through dated directories for all matched inlet and outlet files
for i in range(0, len(F), 1):
    
    # Create a temporary variable of the file information dataframe
    X = F[i]
    
    # If the experiment ID is not present, then it will omit the row for future analysis
    X = X.loc[X['Experiment Set #'] != 'NA'].reset_index().drop(columns = ["index"])
    
    # Only consider non-empty dataframes due to line above potentially deleting all variables
    if len(X) > 0:
        
        # Determine the unique number of experiment sets
        sets = X['Experiment Set #'].nunique()
        
        # Loop that goes through each set
        for j in range(0, sets, 1):
            
            # Add one to the index because Python indexes at 0 and Experiment ID's start at 1
            ID = j+1
            
            # Subset based on experiment ID
            Y = X.loc[X['Experiment Set #'] == ID].reset_index().drop(columns = ["index"])

            # First date value for an experiment set will always be the same as the second sample
            date = Y["Date"][0]

            # Format the date string
            date = datetime.strptime(date, "%Y-%m-%d")
            date = datetime.strftime(date, format = "%Y-%m-%d")
            
            # Print the date and experiment set ID during the loop
            print(date, Y["Time"][0], 'Experiment #:', ID)
            
            # Y is the samples pertaining to an experiment set
            for k in range(0, len(Y), 1):
                
                PF = max(Y['PF'])
                AF = max(Y['AF'])
                SF = max(Y['SF'])
                AIF = max(Y['AIF'])
                IF = round(max(Y['IF']), 1)
                
                # Data
                if (Y["Type"][k] == "Inlet") == True:
                    
                    # Use pd.read_csv() to read in the OPS data
                    Z = pd.read_csv(Y["Path"][k], sep = ',', skip_blank_lines = True, skiprows = 6, encoding = 'latin1')
                    Z = Z.transpose().reset_index()
                    Z.columns = Z.iloc[0]
                    Z = Z.drop([0])

                    # Temporary string of column names
                    tmp_nm = list(Z.columns)

                    # Simple loop to obtain the indices of specific column names
                    # Add one to each because Python index starts at 0
                    for l in range(0, len(tmp_nm), 1):

                        # this is where you specify which columns of APS data you want to keep
                        if tmp_nm[l] == '0.965':
                            start_index = l + 1
                        if tmp_nm[l] == '19.81':
                            end_index = l + 1
                    
                    # Pandas chaining to transpose, change types, and calculate row means
                    # If not clear review the Pandas documentation
                    APS_IN = Z.iloc[:, start_index:end_index].transpose().copy(deep = True).astype("float64")
                    APS_IN["Inlet Mean Concentration"] = APS_IN.astype("float64").mean(axis = 1)
                    APS_IN = APS_IN.reset_index().rename(columns = {0: "Diameter Midpoint"})
                    
                # Data
                if (Y["Type"][k] == "Outlet") == True:
                
                    # Use pd.read_csv() to read in the OPS data
                    Z = pd.read_csv(Y["Path"][k], sep = ',', skip_blank_lines = True, skiprows = 6, encoding = 'latin1')
                    Z = Z.transpose().reset_index()
                    Z.columns = Z.iloc[0]
                    Z = Z.drop([0])

                    tmp_nm = list(Z.columns)

                    # Simple loop to obtain the indices of specific column names
                    # Add one to each because Python index starts at 0
                    for l in range(0, len(tmp_nm), 1):

                        # this is where you specify which columns of APS data you want to keep
                        if tmp_nm[l] == '0.965':
                            start_index = l + 1
                        if tmp_nm[l] == '19.81':
                            end_index = l + 1

                    # Pandas chaining to transpose, change types, and calculate row means
                    # If not clear review the Pandas documentation
                    APS_OUT = Z.iloc[:, start_index:end_index].transpose().copy(deep = True).astype("float64")
                    APS_OUT["Outlet Mean Concentration"] = APS_OUT.astype("float64").mean(axis = 1)
                    APS_OUT = APS_OUT.reset_index().rename(columns = {0: "Diameter Midpoint"})
            
            # Assigment of the values from APS_OUT, APS_IN, and flows to a combined dataframe
            data_df = APS_IN[["Diameter Midpoint", "Inlet Mean Concentration"]].copy(deep = True)
            data_df['Outlet Mean Concentration'] = APS_OUT["Outlet Mean Concentration"]
            data_df['PF'] = PF
            data_df['AF'] = AF
            data_df['SF'] = SF
            data_df['AIF'] = AIF
            data_df['IF'] = round(IF, 1)
            data_df['BF'] = IF - AIF
            data_df['AF:IF Ratio'] = round(AF/IF, 2)

            # Calculate transmission ratio
            data_df['Outlet/Inlet'] = data_df['Outlet Mean Concentration']/data_df['Inlet Mean Concentration']
            
            # Calculate enhancemnet factor
            data_df['IF/SF'] = data_df['IF']/data_df['SF']
            
            # Calculate transmission efficiency
            data_df['Transmission Efficiency'] = data_df['Outlet/Inlet']/data_df['IF/SF']

            # Conversion
            data_df["Diameter Midpoint"] = data_df["Diameter Midpoint"].astype("float64")

            # FILE NAMING
            file_label = date+"_PF"+str(data_df['PF'][0]).replace('.', '_')+"_AF"+str(data_df['AF'][0]).replace('.', '_')+"_results"+".csv"
            file_nm = export_results + file_label

            # Export File
            data_df.to_csv(file_nm)               

            # Size Distribution Plot
            # ----------------------------------------------------------------------- #
            
            # Set plotting style
            plt.style.use('ggplot')

            # Define figure axes and subplots
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (16, 16))

            # Create a title string
            plot_title = "L-PCVI Calibration Results -  " + date + " , PF = " + str(data_df["PF"][0]) + ", AF = " + str(data_df["AF"][0]) + ", SF = " + str(data_df["SF"][0]) + ", IF = " + str(int(data_df["IF"][0])) + ", AIF = " + str(data_df["AIF"][0]) + ", AF:IF Ratio = " + str(data_df["AF:IF Ratio"][0])

            # Change font and type of super title
            #fig.suptitle(plot_title)

            # Indices for plot using list comprehension
            idx = np.asarray([i for i in range(len(data_df['Diameter Midpoint']))])

            # Define width
            width = 0.8

            # Create bars
            bar1 = ax1.bar(x = idx, height = data_df['Inlet Mean Concentration'], width = width, align = "center", alpha = 0.5)
            bar2 = ax1.bar(x = idx, height = data_df['Outlet Mean Concentration'], width = width, align = "center", alpha = 0.5)

            # Plot cosmetics
            ax1.set_title(plot_title)
            ax1.set_xticks(idx)
            ax1.set_xticklabels(data_df['Diameter Midpoint'], rotation = 90)
            ax1.set_xbound(lower = -1, upper = len(data_df['Diameter Midpoint']))
            ax1.text(-0.075, 0.5, r'Mean Concentration $\mathrm{(cm^{-3})}$', rotation = 90, size = 12, weight = 'normal', verticalalignment='center', horizontalalignment='right', transform = ax1.transAxes)

            ax1.legend([bar1, bar2], ['Inlet', 'Outlet'])

            # Transmission Plot
            # ----------------------------------------------------------------------- #

            # Remove NaN's to prevent errors in curve fitting
            data_df = data_df.dropna(thresh = 10)

            # Specify variables used in plotting
            xdata = data_df["Diameter Midpoint"]
            xdata_max = round(max(xdata), 1)
            xdata_min = round(min(xdata), 1)
            ydata = data_df["Transmission Efficiency"]

            # Create a numpy array of diameter midpoints
            idx = np.asarray([i for i in range(len(data_df['Diameter Midpoint']))])
            
            # Plot transmission data
            ax2.scatter(xdata,ydata)
            ax2.set_xlabel(r'Aerodynamic Diameter $\mathrm{(\mu m)}$', size = 12, labelpad = 20)
            ax2.text(-0.075, 0.5, "Transmission Efficiency", rotation = 90, size = 12, weight = 'normal', verticalalignment='center', horizontalalignment='right', transform = ax2.transAxes)
            ax2.set_xticks(idx)
            ax2.set_xbound(lower = 0.5, upper = 20.5)
            ax2.set_xlim(right = (xdata_max + 1))

            # CURVE FITTING
            # ----------------------------------------------------------------------- #

            # Define the sigmoid function
            def sigmoid(x, L ,x0, k, b):
                y = L / (1 + np.exp(-k*(x-x0)))+b
                return (y)

            # Define L
            L = max(data_df['Transmission Efficiency']) # The maximum transmission efficiency

            # Initial guess for curve_fit model
            # REQUIRED guess to prevent RuntimeErrors from occuring constantly
            p0 = [max(ydata), np.median(xdata), 1, min(ydata)]

            try :
                # Fit a sigmoid to the data to idenitfy the cut point
                # Dogbox method is the most accurate however most computationally expensive
                # Absolute sigma is not necessary here
                popt, pcov = curve_fit(sigmoid, xdata, ydata, p0, method = 'dogbox', absolute_sigma = False, maxfev = 1000)
                
                failed = 0

            except RuntimeError:
                print("Error - curve_fit failed")
                
                failed = 1

            # Generate 1000 evenly spaced values between 0 and the maximum midpoint
            # y values are the corresponding model results
            x = np.linspace(xdata_min, xdata_max, 1000)
            y = sigmoid(x, *popt)

            # Calculate sigma values
            sigma = np.sqrt(np.diagonal(pcov))

            # Extract the cut-point
            D50 = popt[1]
            D50_rounded = round(D50, 2)
            
            # Extract the standard deviation corresponding to the cut point
            D50_sigma = sigma[1]
            D50_sigma = round(D50_sigma, 2)
            
            if failed == 0:
                
                # Plot best fit curve
                ax2.plot(x, y, color = 'black', linestyle = '--')

                ax2.plot(D50, sigmoid(D50, *popt), 'x', color = 'blue')
                ax2.errorbar(x = D50, y = sigmoid(D50, *popt), xerr = D50_sigma, color = 'black')

                D50_label = r'Calculated $\mathrm{D_{50} = }$ ' + str(D50_rounded) + r' $\mathrm{\pm}$ ' + str(D50_sigma)

                ax2.legend(["Best Fit Curve", D50_label, "Transmission Efficiencies"])
                
                # Generate a figure label for the filename
                figure_label = date + " PF" + str(data_df['PF'][0]) + " AF" + str(data_df['AF'][0]) + " Ratio" + str(data_df["AF:IF Ratio"][0]) + " Results" + ".png"
                figure_nm = export_plots + figure_label
                
                # Save the figure
                plt.savefig(figure_nm, format = 'png', dpi = 400)

                plt.close()
                
            if failed == 1:

                # Generate a figure label for the filename
                figure_label = date + " PF" + str(data_df['PF'][0]) + " AF" + str(data_df['AF'][0]) + " Ratio" + str(data_df["AF:IF Ratio"][0]) + " Results Failed" + ".png"
                figure_nm = export_plots + figure_label
                
                # Save the figure
                plt.savefig(figure_nm, format = 'png', dpi = 400)

                plt.close()
            
            results['Date'].append(date)
            results['ID'].append(ID)
            results['PF'].append(PF)
            results['AF'].append(AF)
            results['SF'].append(SF)
            results['AIF'].append(AIF)
            results['IF'].append(IF)
            results['AF:IF Ratio'].append(round(AF/IF, 2))
            results['Maximum Transimission Efficiency'].append(round(L, 2))

            if failed == 0:
                if D50 > 0:
                    results['D50'].append(D50_rounded)
                    results['D50 SD'].append(D50_sigma)
                else:
                    results['D50'].append("Failed")
                    results['D50 SD'].append("Failed")
            else:
                results['D50'].append("Failed")
                results['D50 SD'].append("Failed")
        

2021-08-10 1250 Experiment #: 1
2021-08-10 1335 Experiment #: 2
2021-08-10 1348 Experiment #: 3
2021-08-10 1423 Experiment #: 4
2021-08-10 1449 Experiment #: 5
2021-08-10 1456 Experiment #: 6
2021-08-10 1522 Experiment #: 7
2021-08-10 1535 Experiment #: 8
2021-08-10 1550 Experiment #: 9
2021-08-10 1558 Experiment #: 10
2021-08-10 1611 Experiment #: 11
2021-09-09 1258 Experiment #: 1
2021-09-09 1304 Experiment #: 2
2021-09-09 1310 Experiment #: 3
2021-09-09 1312 Experiment #: 4
2021-09-09 1322 Experiment #: 5
2021-09-09 1327 Experiment #: 6
2021-09-01 1629 Experiment #: 1
2021-09-01 1646 Experiment #: 2
2021-09-01 1650 Experiment #: 3
2021-09-01 1700 Experiment #: 4
2021-09-01 1709 Experiment #: 5
2021-09-01 1719 Experiment #: 6
2021-07-31 1722 Experiment #: 1
2021-07-31 1738 Experiment #: 2
2021-07-31 1750 Experiment #: 3
2021-07-31 1804 Experiment #: 4
2021-07-31 1821 Experiment #: 5
2021-07-31 1834 Experiment #: 6
2021-07-31 1851 Experiment #: 7
2021-07-31 1857 Experiment #: 8
2021-0

In [10]:
# Dataframe conversion
results_df = pd.DataFrame.from_dict(results, orient = 'index').transpose().sort_values("PF", ascending = False).reset_index().drop(columns = ["index"])

# Find the current time for exporting data
sys_time = str(datetime.now())[0:10]
sys_time

# Filename
file_nm = export_results + "LPCVI_results_summary_" + sys_time + ".csv"

# Export as a csv
results_df.to_csv(file_nm)

In [11]:
results_df

,Date,ID,PF,AF,SF,AIF,IF,AF:IF Ratio,Maximum Transimission Efficiency,D50,D50 SD
0,2021-07-28,8,87,7,6.5,5,86.5,0.08,0.05,7.5,0.18
1,2021-07-28,6,85,7,6.5,5,84.5,0.08,0.05,8.55,0.39
2,2021-07-28,7,85,9,6.5,5,82.5,0.11,0.07,9.91,0.59
3,2021-07-28,4,83,7,6.5,5,82.5,0.08,0.5,8.32,0.24
4,2021-07-28,5,83,9,6.5,5,80.5,0.11,0.15,9.7,0.27
...,...,...,...,...,...,...,...,...,...,...,...
103,2021-08-04,2,33,7.9,1,5,26.1,0.3,0.13,13.53,0.41
104,2021-08-04,3,33,7.9,6.5,5,31.6,0.25,0.82,6.86,0.44
105,2021-08-04,1,31,7.5,1,5,24.5,0.31,0.12,12.59,0.5
106,2021-07-22,1,30,4.48,6.5,5,32,0.14,0.78,5.49,0.1
